# Advent of Code 2024

## Day 1

### Part 1

In [8]:
with open('inputs/day1.txt') as f:
    s = f.read().split('\n')[:-1]

left = sorted([int(line.split('   ')[0]) for line in s])
right = sorted([int(line.split('   ')[1]) for line in s])

sum([abs(left[i] - right[i]) for i in range(len(left))])

2344935

### Part 2

In [10]:
frequency = {n: right.count(n) for n in left}

sum([n*frequency[n] for n in frequency])

27647262

## Day 2

### Part 1

In [17]:
with open('inputs/day2.txt') as f:
    s = f.read().split('\n')[:-1]


def is_safe(levels):
    direction = levels[1] - levels[0]
    for i in range(1, len(levels)):
        d = levels[i] - levels[i-1]
        if direction * d < 0 or abs(d) < 1 or abs(d) > 3:
            return False
    return True


report = [[int(level) for level in line.split(' ')] for line in s]
sum([1 if is_safe(levels) else 0 for levels in report])

516

### Part 2

In [19]:
def remove_level(levels, i):
    return levels[:i] + levels[i+1:]


def is_safe_tolerate(levels):
    if is_safe(levels):
        return True
    for i in range(len(levels)):
        if is_safe(remove_level(levels, i)):
            return True
    return False


sum([1 if is_safe_tolerate(levels) else 0 for levels in report])

561

## Day 3

### Part 1

In [31]:
import re

with open('inputs/day3.txt') as f:
    s = f.read()[:-1]


def mul(command):
    num = [int(i) for i in command[4:-1].split(',')]
    return num[0]*num[1]


reg = re.compile(r'mul\(\d{1,3},\d{1,3}\)')
sum([mul(i) for i in reg.findall(s)])

189527826

### Part 2

In [36]:
reg = re.compile(r'mul\(\d{1,3},\d{1,3}\)|do\(\)|don\'t\(\)')

a = 0
enabled = True
for comm in reg.findall(s):
    if comm == 'do()':
        enabled = True
    elif comm == 'don\'t()':
        enabled = False
    elif enabled:
        a += mul(comm)

a

63013756

## Day 4

### Part 1

In [23]:
with open('inputs/day4.txt') as f:
    s = f.read().split('\n')[:-1]


def transpose(s):
    return [''.join([s[y][x] for y in range(len(s))]) for x in range(len(s[0]))]


def diagonal_words(s, min_l):
    words = []
    for y in range(len(s)):
        for x in range(len(s[0])):
            for (dx,dy) in [(1,1),(1,-1),(-1,1),(-1,-1)]:
                if x + (min_l-1)*dx in range(len(s[0])) and y + (min_l-1)*dy in range(len(s)):
                    words.append(''.join([s[y+i*dy][x+i*dx] for i in range(min_l)]))
    return words


search = 'XMAS'
a = 0

for line in s:
    a += line.count(search)
    a += line.count(search[::-1])

for line in transpose(s):
    a += line.count(search)
    a += line.count(search[::-1])

a += diagonal_words(s, len(search)).count(search)

a

2583

### Part 2

In [24]:
search = 'MAS'

def check_cross_word(x,y,s):
    x_in_range = x in range(1, len(s[0])-1)
    y_in_range = y in range(1, len(s)-1)
    if not x_in_range or not y_in_range:
        return False
    center_ok = s[y][x] == search[1]
    diag_up_left    = s[y-1][x-1] == search[0] and s[y+1][x+1] == search[2]
    diag_up_right   = s[y-1][x+1] == search[0] and s[y+1][x-1] == search[2]
    diag_down_left  = s[y+1][x-1] == search[0] and s[y-1][x+1] == search[2]
    diag_down_right = s[y+1][x+1] == search[0] and s[y-1][x-1] == search[2]
    return center_ok and (diag_up_left or diag_down_right) and (diag_up_right or diag_down_left)


len([(x,y) for y in range(len(s)) for x in range(len(s[0])) if check_cross_word(x,y,s)])

1978

## Day 5

### Part 1

In [27]:
with open('inputs/day5.txt') as f:
    s = f.read()[:-1].split('\n\n')


def in_order(pages, rules):
    visited = set()
    for page in pages:
        visited.add(page)
        related_rules = rules.get(page, [])
        for rule in related_rules:
            if rule in pages and rule not in visited:
                return False
    return True


def middle(pages):
    return pages[len(pages)//2]


rules = {}
for rule in s[0].split('\n'):
    (x,y) = tuple(int(i) for i in rule.split('|'))
    preceding = rules.get(y, [])
    preceding.append(x)
    rules[y] = preceding

pages = [[int(i) for i in line.split(',')] for line in s[1].split('\n')]

sum([middle(p) for p in pages if in_order(p, rules)])

7365

### Part 2

In [28]:
def filtered_rules(page, pages, rules, visited):
    return [r for r in rules.get(page, []) if r in pages and r not in visited]


def order(pages, rules):
    result = []
    while len(result) < len(pages):
        relevant_rules = {p: filtered_rules(p, pages, rules, result) for p in pages if p not in result}
        next_page = [r for r in relevant_rules if len(relevant_rules[r]) == 0][0]
        result.append(next_page)
    return result

sum([middle(order(p, rules)) for p in pages if not in_order(p, rules)])

5770

## Day 6

### Part 1

In [107]:
with open('inputs/day6.txt') as f:
    s = f.read().split('\n')[:-1]


directions = [(0,-1),(1,0),(0,1),(-1,0)]
obst = set()
for y in range(len(s)):
    for x in range(len(s[0])):
        if s[y][x] == '#':
            obst.add((x,y))
        if s[y][x] == '^':
            start = (x,y)


def move_guard(pos, facing, obst):
    (x,y) = pos
    (dx,dy) = directions[facing]
    if (x+dx, y+dy) not in obst:
        return ((x+dx, y+dy), facing)
    return (pos, (facing+1)%4)


def walk(start, facing, obst):
    visited = set()
    guard = start
    while guard[0] in range(len(s[0])) and guard[1] in range(len(s)) and (guard, facing) not in visited:
        visited.add((guard, facing))
        (guard, facing) = move_guard(guard, facing, obst)
    return visited
    

visited = walk(start, 0, obst)
len({v[0] for v in visited})

4977

### Part 2

In [108]:
def has_loop(visited):
    return len([((x,y), f) for ((x,y), f) in visited if x+directions[f][0] not in range(len(s[0])) or y+directions[f][1] not in range(len(s))]) == 0


looping = set()
for (guard, facing) in visited:
    new_obst = (guard[0]+directions[facing][0], guard[1]+directions[facing][1])
    if new_obst not in obst and new_obst[0] in range(len(s[0])) and new_obst[1] in range(len(s)):
        new_obsts = {o for o in obst}
        new_obsts.add(new_obst)
        new_visited = walk(start, 0, new_obsts)
        if has_loop(new_visited):
            looping.add(new_obst)

len(looping)

1729

## Day 7

### Part 1

In [11]:
with open('inputs/day7.txt') as f:
    s = f.read().split('\n')[:-1]

equations = [(int(line.split(': ')[0]), [int(i) for i in line.split(': ')[1].split(' ')]) for line in s]


def options(current, nxt):
    return [current+nxt, current*nxt]


def has_solution(target, current, remaining):
    if len(remaining) == 0:
        return current == target
    if current > target:
        return False
    opt = options(current, remaining[0])
    solvable = False
    for o in opt:
        solvable |= has_solution(target, o, remaining[1:])
    return solvable

sum([eq[0] for eq in equations if has_solution(eq[0], 0, eq[1])])

1038838357795

### Part 2

In [12]:
def options(current, nxt):
    return [current+nxt, current*nxt, int(str(current)+str(nxt))]

sum([eq[0] for eq in equations if has_solution(eq[0], 0, eq[1])])

254136560217241

### Day 8

### Part 1

In [17]:
with open('inputs/day8.txt') as f:
    s = f.read().split('\n')[:-1]


def antinodes(antennas, width, height):
    anti = set()
    for a1 in antennas:
        for a2 in antennas:
            if a1 != a2:
                dx = a1[0]-a2[0]
                dy = a1[1]-a2[1]
                anti.add((a1[0]+dx,a1[1]+dy))
                anti.add((a2[0]-dx,a2[1]-dy,))
    return {a for a in anti if a[0] in range(width) and a[1] in range(height)}


width = len(s[0])
height = len(s)
antennas = {}
for y in range(height):
    for x in range(width):
        if s[y][x] != '.':
            a = antennas.get(s[y][x], [])
            a.append((x,y))
            antennas[s[y][x]] = a

anti = set()
for a in antennas:
    anti.update(antinodes(antennas[a], width, height))

len(anti)

361

### Part 2

In [18]:
def antinodes(antennas, width, height):
    anti = {a for a in antennas}
    for a1 in antennas:
        for a2 in antennas:
            if a1 != a2:
                dx = a1[0]-a2[0]
                dy = a1[1]-a2[1]
                i = 1
                while a1[0]+i*dx in range(width) and a1[1]+i*dy in range(height):
                    anti.add((a1[0]+i*dx,a1[1]+i*dy))
                    i += 1
                i = 1
                while a1[0]+i*dx in range(width) and a1[1]+i*dy in range(height):
                    anti.add((a2[0]-i*dx,a2[1]-i*dy))
                    i += 1
    return {a for a in anti if a[0] in range(width) and a[1] in range(height)}

anti = set()
for a in antennas:
    anti.update(antinodes(antennas[a], width, height))

len(anti)

1249

## Day 9

### Part 1

In [71]:
with open('inputs/day9.txt') as f:
    s = f.read()[:-1]

disk = []
file = True
file_id = 0
file_blocks = 0
disk_idx = 0
for i in s:
    disk.append((disk_idx, disk_idx+int(i), file, file_id if file else None))
    if file:
            file_blocks += int(i)
            file_id += 1
    file = not file
    disk_idx += int(i)

a = 0
fwd_idx = 0
bwd_idx = disk[-1][1]-1
fwd_block = 0
bwd_block = len(disk)-1
while fwd_idx < file_blocks:
    if disk[fwd_block][2]:
        a += fwd_idx*disk[fwd_block][3]
    else:
        a += fwd_idx*disk[bwd_block][3]
        bwd_idx -= 1
        if bwd_idx < disk[bwd_block][0]:
            bwd_block -= 2
            bwd_idx = disk[bwd_block][1]-1
    fwd_idx += 1
    while fwd_idx >= disk[fwd_block][1]:
        fwd_block += 1

a

6154342787400

### Part 2

In [72]:
def fits(file, into):
    return file[1]-file[0] <= into[1]-into[0]


def move(disk, idx, target):
    empty_block = disk[target]
    file_block = disk[idx]
    new_file_block = (empty_block[0], empty_block[0]+(file_block[1]-file_block[0]), True, file_block[3])
    new_empty_block = (file_block[0], file_block[1], False, None)
    disk[target] = new_file_block
    
    disk[idx] = new_empty_block
    if new_file_block[1] < empty_block[1]:
        disk.insert(target+1, (new_file_block[1], empty_block[1], False, None))


def find_space(disk, idx):
    for i in range(idx):
        if not disk[i][2] and fits(disk[idx], disk[i]):
            return i


def defrag(disk):
    defragged = [d for d in disk]
    for idx in range(len(defragged)-1, -1, -1):
        if defragged[idx][2]:
            target = find_space(defragged, idx)
            if target is not None:
                move(defragged, idx, target)
    return defragged


defragged = defrag(disk)
sum([block[3]*sum(range(block[0], block[1])) for block in defragged if block[2]])

6183632723350

## Day 10

### Part 1

In [110]:
with open('inputs/day10.txt') as f:
    s = f.read().split('\n')[:-1]

grid = {(x,y): int(s[y][x]) for y in range(len(s)) for x in range(len(s[0])) if s[y][x] != '.'}
starts = {c for c in grid if grid[c] == 0}


def reachable(coord, grid):
    (x,y) = coord
    return [(x+dx,y+dy) for (dx,dy) in [(1,0),(-1,0),(0,1),(0,-1)] if (x+dx,y+dy) in grid and grid[(x+dx,y+dy)] == grid[coord]+1]


def score(grid, start):
    visited = set()
    queue = {start}
    while len(queue) > 0:
        current = queue.pop()
        visited.add(current)
        for nxt in reachable(current, grid):
            queue.add(nxt)
    return len([c for c in visited if grid[c] == 9])

sum([score(grid, start) for start in starts])

786

### Part 2

In [111]:
ratings = {c: 1  if grid[c] == 0 else 0 for c in grid}
for height in range(1,10):
    for (x,y) in ratings:
        if grid[(x,y)] == height-1:
            for (dx,dy) in [(1,0),(-1,0),(0,1),(0,-1)]:
                if (x+dx,y+dy) in grid and grid[(x+dx,y+dy)] == height:
                    ratings[(x+dx,y+dy)] = ratings[(x+dx,y+dy)] + ratings[(x,y)]

sum([ratings[c] for c in ratings if grid[c] == 9])

1722

## Day 11

### Part 1

In [42]:
with open('inputs/day11.txt') as f:
    s = f.read()[:-1]


def blink_single(stone):
    if stone == 0:
        return [1]
    if len(str(stone)) % 2 == 0:
        string = str(stone)
        first = string[:len(string)//2]
        second = string[len(string)//2:]
        return [int(first), int(second)]
    return [stone*2024]


def blink(stones):
    new_stones = {}
    for stone in stones:
        for st in blink_single(stone):
            new_stones[st] = new_stones.get(st, 0) + stones[stone]
    return new_stones


stones = {int(n):1 for n in s.split(' ')}

for _ in range(25):
    stones = blink(stones)

sum(stones.values())

212655

### Part 2

In [47]:
stones = {int(n):1 for n in s.split(' ')}

for _ in range(75):
    stones = blink(stones)

sum(stones.values())

253582809724830

## Day 12

### Part 1

In [36]:
with open('inputs/day12.txt') as f:
    s = f.read().split('\n')[:-1]


def neighbour_coords(coord):
    (x,y) = coord
    return [(x+dx,y+dy) for (dx,dy) in [(1,0),(-1,0),(0,1),(0,-1)]]


def neighbours(coord, grid):
    return [c for c in neighbour_coords(coord) if c in grid]


def fill_region(start, grid):
    visited = {start}
    queue = {start}
    while len(queue) > 0:
        current = queue.pop()
        nxt = [n for n in neighbours(current, grid) if grid[n] == grid[start] and n not in visited]
        visited.update(nxt)
        queue.update(nxt)
    return visited


def next_unvisited(all_nodes, visited):
    for c in all_nodes:
        if c not in visited:
            return c


def identify_regions(grid):
    regions = {}
    visited = set()
    region_id = 0
    while len(visited) < len(grid):
        current = next_unvisited(grid, visited)
        area = fill_region(current, grid)
        regions[region_id] = area
        visited.update(area)
        region_id += 1
    return regions


def identify_sides(region):
    return {(coord, c) for coord in region for c in neighbour_coords(coord) if c not in region}


def area(region):
    return len(region)


def perimeter(region):
    return len(identify_sides(region))


grid = {(x,y): s[y][x] for y in range(len(s)) for x in range(len(s[0]))}
regions = identify_regions(grid)

sum([area(regions[r])*perimeter(regions[r]) for r in regions])

1424472

### Part 2

In [37]:
def neighbouring_sides(side, sides):
    ((x1,y1),(x2,y2)) = side
    return [((x1+dx,y1+dy),(x2+dx,y2+dy)) for (dx,dy) in [(1,0),(-1,0),(0,1),(0,-1)] if ((x1+dx,y1+dy),(x2+dx,y2+dy)) in sides]


def traverse_side(start, sides):
    visited = {start}
    queue = {start}
    while len(queue) > 0:
        current = queue.pop()
        nxt = [n for n in neighbouring_sides(current, sides) if n not in visited]
        visited.update(nxt)
        queue.update(nxt)
    return visited      


def region_sides(region):
    side_sections = identify_sides(region)
    sides = {}
    visited = set()
    side_id = 0
    while len(visited) < len(side_sections):
        current = next_unvisited(side_sections, visited)
        side = traverse_side(current, side_sections)
        sides[side_id] = side
        visited.update(side)
        side_id += 1
    return sides
    
    
def side_count(region):
    return len(region_sides(region))


sum([area(regions[r])*side_count(regions[r]) for r in regions])

870202

## Day 13

### Part 1

In [89]:
with open('inputs/day13.txt') as f:
    s = f.read()[:-1].split('\n\n')


def parse_button(line):
    parts = line.replace(',', '+').split('+')
    return (int(parts[1]), int(parts[-1]))


def parse_prize(line):
    parts = line.replace(',', '=').split('=')
    return (int(parts[1]), int(parts[-1]))
    

def parse_machine(s):
    lines = s.split('\n')
    return (parse_button(lines[0]),parse_button(lines[1]),parse_prize(lines[2]))


def solve(machine):
    ((ax,ay), (bx,by), (px,py)) = machine
    if ay*bx-ax*by != 0:
        b = (ay*px-ax*py) / (ay*bx-ax*by)
        a = (px-b*bx) / ax
        if a == int(a) and b == int(b) and a > 0 and b > 0:
            return (int(a),int(b))


def tokens(machine):
    buttons = solve(machine)
    if buttons is not None:
        a,b = buttons
        return 3*a+b
    return 0

machines = [parse_machine(machine) for machine in s]
sum([tokens(machine) for machine in machines])

39996

### Part 2

In [90]:
increase = 10000000000000
machines = [parse_machine(machine) for machine in s]
machines = [(a,b,(px+increase, py+increase)) for (a,b,(px,py)) in machines]

sum([tokens(machine) for machine in machines])

73267584326867

## Day 14

### Part 1

In [116]:
with open('inputs/day14.txt') as f:
    s = f.read().split('\n')[:-1]


def parse_line(line):
    return tuple(tuple(int(i) for i in vec.split('=')[-1].split(',')) for vec in line.split(' '))


robots = [parse_line(line) for line in s]
width = 101
height = 103


def pos_after(p,v,t):
    (px,py) = p
    (vx,vy) = v
    return (((px+t*vx)%width, (py+t*vy)%height), v)


def move_all(robots, t):
    return [pos_after(p,v,t) for (p,v) in robots]


robots_final = move_all(robots,100)
separator_x = width//2 
separator_y = height//2

quad_1 = len([(px,py) for ((px,py),_) in robots_final if px in range(separator_x) and py in range(separator_y)])
quad_2 = len([(px,py) for ((px,py),_) in robots_final if px in range(separator_x+1, width) and py in range(separator_y)])
quad_3 = len([(px,py) for ((px,py),_) in robots_final if px in range(separator_x) and py in range(separator_y+1, height)])
quad_4 = len([(px,py) for ((px,py),_) in robots_final if px in range(separator_x+1, width) and py in range(separator_y+1, height)])

quad_1 * quad_2 * quad_3 * quad_4

229839456

### Part 2

In [120]:
def has_tree(robots):
    robots_p = [r[0] for r in robots]
    robots_t = [r for r in robots_p]
    for d in range(1,8):
        robots_t = [(px,py) for (px,py) in robots_t if (px+d,py) in robots_p]
    return len(robots_t) > 0


t = 0
while not has_tree(move_all(robots,t)):
    t += 1

t

7138

## Day 15

### Part 1

In [90]:
with open('inputs/day15.txt') as f:
    s = f.read()[:-1].split('\n\n')


area = s[0].split('\n')
inst = s[1].replace('\n', '')
directions = {'<': (-1,0), '>': (1,0), 'v': (0,1), '^': (0,-1)}

grid = set()
boxes = set()
for y in range(len(area)):
    for x in range(len(area[0])):
        if area[y][x] != '#':
            grid.add((x,y))
            if area[y][x] == 'O':
                boxes.add((x,y))
            if area[y][x] == '@':
                robot = (x,y)


def move(coord, direction, grid):
    (x,y) = coord
    (dx,dy) = direction
    nxt = (x+dx,y+dy)
    if nxt in grid:
        return nxt
    return coord
    


def push(coord, direction, boxes, grid):
    (x,y) = coord
    (dx,dy) = direction
    boxes_new = {b for b in boxes}
    if coord in boxes:
        nxt = (x+dx,y+dy)
        if nxt in grid:
            boxes_new = push(nxt,direction,boxes,grid)
            if nxt not in boxes_new:
                boxes_new.remove(coord)
                boxes_new.add(nxt)
    return boxes_new


def gps(coord):
    (x,y) = coord
    return 100*y+x


current = robot
boxes_current = {b for b in boxes}
for i in inst:
    d = directions[i]
    nxt = move(current, d, grid)
    if nxt != current:
        if nxt not in boxes_current:
            current = nxt
        else:
            boxes_nxt = push(nxt, d, boxes_current, grid)
            if boxes_current != boxes_nxt:
                current = nxt
            boxes_current = boxes_nxt


sum([gps(box) for box in boxes_current])

1471826

### Part 2

In [96]:
grid = set()
boxes = set()
for y in range(len(area)):
    for x in range(len(area[0])):
        if area[y][x] != '#':
            grid.add((2*x,y))
            grid.add((2*x+1,y))
            if area[y][x] == 'O':
                boxes.add(((2*x,y),(2*x+1,y)))
            if area[y][x] == '@':
                robot = (2*x,y)


def find_in(coord,boxes):
    for box in boxes:
        if coord in box:
            return box


def push(coord, direction, boxes, grid):
    (dx,dy) = direction
    boxes_new = {b for b in boxes}
    box = find_in(coord, boxes)
    if box is not None:
        nxt = ((box[0][0]+dx, box[0][1]+dy),(box[1][0]+dx, box[1][1]+dy))
        if nxt[0] in grid and nxt[1] in grid:
            nxt_l = find_in(nxt[0], boxes)
            nxt_r = find_in(nxt[1], boxes)
            boxes_new_l = None
            boxes_new_r = None
            if nxt_l is not None and nxt_l != box:
                boxes_new_l = push(nxt[0],direction,boxes_new,grid)
            if nxt_r is not None and nxt_r != box:
                boxes_new_r = push(nxt[1],direction,boxes_new,grid)
            left_ok = boxes_new_l is None or find_in(nxt[0], boxes_new_l) is None
            right_ok = boxes_new_r is None or find_in(nxt[1], boxes_new_r) is None
            if left_ok and right_ok:
                if nxt_l is not None and nxt_l != box:
                    boxes_new = push(nxt[0],direction,boxes_new,grid)
                if nxt_r is not None and nxt_r != box:
                    boxes_new = push(nxt[1],direction,boxes_new,grid)
                boxes_new.remove(box)
                boxes_new.add(nxt)
    return boxes_new


def gps(box):
    ((x,y),_) = box
    return 100*y+x


current = robot
boxes_current = {b for b in boxes}
for i in inst:
    d = directions[i]
    nxt = move(current, d, grid)
    if nxt != current:
        if find_in(nxt, boxes_current) is None:
            current = nxt
        else:
            boxes_nxt = push(nxt, d, boxes_current, grid)
            if boxes_current != boxes_nxt:
                current = nxt
            boxes_current = boxes_nxt


sum([gps(box) for box in boxes_current])

1457703

## Day 16

### Part 1

In [135]:
with open('inputs/day16.txt') as f:
    s = f.read()[:-1].split('\n')

grid = {(x,y): s[y][x] for x in range(len(s[0])) for y in range(len(s)) if s[y][x] != '#'}
directions = [(1,0),(0,1),(-1,0),(0,-1)]
start = [c for c in grid if grid[c] == 'S'][0]
end = [c for c in grid if grid[c] == 'E'][0]


def moves(coord, direction, grid):
    (x,y) = coord
    (dx,dy) = directions[direction]
    options = [(coord, (direction+1)%4, 1000), (coord, (direction-1)%4, 1000)]
    fwd = (x+dx, y+dy)
    if fwd in grid:
        options.append((fwd, direction, 1))
    return options


scores = {(start, 0): 0}
queue = {(start, 0)}
while len(queue) > 0:
    (current, d) = queue.pop()
    current_score = scores[(current, d)]
    for (nxt_coord, nxt_d, cost) in moves(current, d, grid):
        if (nxt_coord, nxt_d) not in scores or current_score+cost < scores[(nxt_coord, nxt_d)]:
            scores[(nxt_coord, nxt_d)] = current_score+cost
            queue.add((nxt_coord, nxt_d))


best = min({(c,d): scores[(c,d)] for (c,d) in scores if grid[c] == 'E'}.values())
best

106512

### Part 2

In [139]:
scores = {(start, 0): 0}
queue = {(start, 0)}
previous = {(start, 0): set()}
while len(queue) > 0:
    (current, d) = queue.pop()
    current_score = scores[(current, d)]
    prev = previous[(current, d)]
    for (nxt_coord, nxt_d, cost) in moves(current, d, grid):
        if (nxt_coord, nxt_d) in scores and current_score+cost == scores[(nxt_coord, nxt_d)]:
            previous[(nxt_coord, nxt_d)].add((current, d))
        if (nxt_coord, nxt_d) not in scores or current_score+cost < scores[(nxt_coord, nxt_d)]:
            scores[(nxt_coord, nxt_d)] = current_score+cost
            queue.add((nxt_coord, nxt_d))
            previous[(nxt_coord, nxt_d)] = {(current, d)}


queue = {(c,d) for (c,d) in previous if grid[c] == 'E' and scores[(c,d)] == best}
visited = set()
while len(queue) > 0:
    current = queue.pop()
    if current not in visited:
        visited.add(current)
        queue.update(previous[current])

tiles = {c for (c,_) in visited}
len(tiles)

563